In [28]:
# Importing Dependencies
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

## EXTRACT AND TRANSFORM

### The below is the first raw Dataset which has three columns viz:
#### 1. movieid: Unique Movie ID for each movie
#### 2. title: The Movie Title and the Release year in Round Brackets
#### 3. genres: The Movie genre

In [2]:
csv_path = "Resources/movies.csv"
movies_df = pd.read_csv(csv_path)
movies_df.dropna()
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# Transformation of the Movies Dataset
csv_file = "Resources/movies.csv"
movies_df = pd.read_csv(csv_file)
movies_df.columns = ['movieid','title','genres']

# Splitting the column 'title' into two columns 'Title' and 'Release Year'
new = movies_df["title"].str.rsplit("(", n = 1, expand = True)
movies_df["Title"] = new[0]
movies_df["Release Year"] = new[1]
movies_df = movies_df.drop(['title'], axis=1)
movies_df.columns = ['movieid', 'genres', 'title', 'release_year']
movies_df = movies_df.dropna()
movies_df['release_year'] = movies_df.release_year.str.replace(')', '')
movies_df.head()

,movieid,genres,title,release_year
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995


### The below is the second raw Dataset which has three columns viz:
####    1. movieid: Unique Movie ID for each movie.
####    2. imdbid: Unique ID for the movie in the IMDB database.
####    3. tmdbid: Unique ID for the movie in the TheMovieDB database.

In [4]:
csv_path1 = "Resources/links.csv"
links_df1 = pd.read_csv(csv_path1)
links_df1.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


#### In this Dataset, we are adding two columns to give the actual IMDB and TMDB movie links for each movie. We are generating this link by appending the IMDB/TMDB links to the imdbid/tmdbid respectively for each movie.

### For IMDB Link:
####    The IMDB ID in the actual hyperlink requires a 7 digit ID. So in the dataset, wherever we have 6 digits in the 'imdbid' column, we are appending '0' before the id in the link. For a 7 digit 'imdbid', no changes are required.
    
### For TMDB Link:
####    The TMDB ID in the actual hyperlink requires a 6 digit ID. So in the dataset, wherever we have less than 6 digits in the 'tmdbid' column, we are appending  a character '0' (6 - number of digits) times before the id in the link. For a 6 digit 'tmdbid', no changes are required.

In [5]:
# Transformation of the Movie Links Dataset
csv_file = "Resources/links.csv"
links_df = pd.read_csv(csv_file)
links_df.columns = ['movieid','imdbid','tmdbid']

In [8]:
# Adding new columns to Create IMDB link strings
links_df['imdb_link'] = links_df.apply(lambda row: "https://www.imdb.com/title/tt" + "0"*(7 - len(str(row.imdbid))) + str(row.imdbid), axis = 1)

# Dropping Blank Columns
links_df = links_df.dropna()

# Changing the Data Type of the Pandas Column
links_df['tmdbid'] = links_df['tmdbid'].astype(int)

# Adding new columns to Create TMDB link strings
links_df['tmdb_link'] = links_df.apply(lambda row: "https://www.themoviedb.org/movie/" + "0"*(6 - len(str(row.tmdbid))) + str(row.tmdbid), axis = 1)

links_df.head()

,movieid,imdbid,tmdbid,imdb_link,tmdb_link
0,1,114709,862,https://www.imdb.com/title/tt0114709,https://www.themoviedb.org/movie/000862
1,2,113497,8844,https://www.imdb.com/title/tt0113497,https://www.themoviedb.org/movie/008844
2,3,113228,15602,https://www.imdb.com/title/tt0113228,https://www.themoviedb.org/movie/015602
3,4,114885,31357,https://www.imdb.com/title/tt0114885,https://www.themoviedb.org/movie/031357
4,5,113041,11862,https://www.imdb.com/title/tt0113041,https://www.themoviedb.org/movie/011862


### The below is the third raw Dataset which has three columns viz:
####    1. movieid: Unique Movie ID for each movie.
####    2. userid: Unique User ID which gives rating to the movie.
####    3. rating: The Rating for each movie
####    4. timestamp: The actual timestamp when the User has given the rating for the movie.

In [9]:
csv_path = "Resources/ratings.csv"
ratings_df = pd.read_csv(csv_path)
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


#### This Data gives the ratings provided by user for every movie. Each movie may be rated by multiple users. Also, one user can rate multiple movies. We are focusing on the ratings for each movie. So, we are calculating the mean of all ratings for each movie and dropping the column 'userId'.

In [10]:
# Transformation of the Ratings Dataset
csv_file = "Resources/ratings.csv"
ratings_df = pd.read_csv(csv_file)

# Grouping the Dataset by 'movieid' and calculating the Mean
avg_ratings_df = ratings_df.groupby(['movieId'], as_index=False).mean()
avg_ratings_df.head()

,movieId,userId,rating,timestamp
0,1,141939.237859,3.886649,1.128094e+09
1,2,142395.293962,3.246583,1.105961e+09
2,3,140371.877575,3.173981,9.692933e+08
3,4,140527.990632,2.874540,9.405874e+08
4,5,141254.322735,3.077291,9.970812e+08


In [11]:
# New ratings data with select columns
new_avg_ratings_df = avg_ratings_df[['movieId', 'rating']].copy()
new_avg_ratings_df.columns = ['movieid','rating']
new_avg_ratings_df.head()

,movieid,rating
0,1,3.886649
1,2,3.246583
2,3,3.173981
3,4,2.874540
4,5,3.077291


### The below is the fourth raw Dataset which has three columns viz:
####    1. movieid: Unique Movie ID for each movie.
####    2. userid: Unique User ID which gives rating to the movie.
####    3. tag: The tag provided by the user for each movie.
####    4. timestamp: The actual timestamp when the User has given the tag for the movie.

In [12]:
csv_file = "Resources/tags.csv"
tags_df = pd.read_csv(csv_file)
tags_df.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


### This Data gives the tags provided by user for every movie. Each movie may have tags provided by multiple users. Also, one user can provide tags to multiple movies. 

### Here, we need to collate all the tags provided by the users for each movie. We are initially collating all the tags for each movie into a Set data structure and then converting the Set into a comma-separated string.

In [13]:
# Transformation of the Tags Dataset
new_tags_df = tags_df[['movieId', 'tag']].copy()
new_tags_df.columns = ['movieid','tag']

# Grouping Each Movie ID and Concatenating the Tags for each movie (Converts the tag to a set)
new_tags_df = new_tags_df.set_index('movieid')
new_tags_df = new_tags_df.groupby('movieid').agg(lambda x: set(x.tag)).reset_index().head()
new_tags_df.head()

,movieid,tag
0,1,"{boy next door, family, disney, ya boy, BD-Vid..."
1,2,"{see also:Zathura, giant insect, adapted from:..."
2,3,"{moldy, comedinha de velhinhos engraÃƒÂ§ada, N..."
3,4,"{based on novel or book, interracial relations..."
4,5,"{Touching, family, Steve Martin, it thought it..."


In [14]:
# Converting the 'tag' column of type 'set' into a string separated by comma
new_tags_df = new_tags_df.set_index('movieid')
new_tags_df['new_tags'] = new_tags_df['tag'].apply(lambda x: ', '.join(x))
new_tags_df.reset_index()
new_tags_df = new_tags_df.drop(['tag'], axis=1)
new_tags_df = new_tags_df.reset_index()
new_tags_df.columns = ['movieid','tag']
new_tags_df.head()

,movieid,tag
0,1,"boy next door, family, disney, ya boy, BD-Vide..."
1,2,"see also:Zathura, giant insect, adapted from:b..."
2,3,"moldy, comedinha de velhinhos engraÃƒÂ§ada, NO..."
3,4,"based on novel or book, interracial relationsh..."
4,5,"Touching, family, Steve Martin, it thought it ..."


### Connect to local database 

### Now, we have the Pandas DataFrames from the four datasets (Movies, Links, Ratings and Tags). We are now creating a SQLITE Database.

### Once the Database is created, we create corresponding Tables for each DataFrame

In [17]:
DB_Name = 'Movies_DB'
engine = create_engine(f'sqlite:///{DB_Name}')

In [18]:
# Create Table 'Movies' within the 'Movies_DB' database
engine.execute('CREATE TABLE "Movies" ('
               'movieid VARCHAR NOT NULL,'
               'genres VARCHAR NOT NULL,'
               'title VARCHAR NOT NULL,'
               'release_year VARCHAR NOT NULL,'
               'PRIMARY KEY (movieid));')

# Create Table 'Links' within the 'Movies_DB' database
engine.execute('CREATE TABLE "Links" ('
               'movieid VARCHAR NOT NULL,'
               'imdbid VARCHAR NOT NULL,'
               'tmdbid VARCHAR NOT NULL,'
               'imdb_link VARCHAR NOT NULL,'
               'tmdb_link VARCHAR NOT NULL,'
               'PRIMARY KEY (movieid));')

# Create Table 'Ratings' within the 'Movies_DB' database
engine.execute('CREATE TABLE "Ratings" ('
               'movieid VARCHAR NOT NULL,'
               'rating FLOAT NOT NULL,'
               'PRIMARY KEY (movieid));')

# Create Table 'Tags' within the 'Movies_DB' database
engine.execute('CREATE TABLE "Tags" ('
               'movieid VARCHAR NOT NULL,'
               'tag VARCHAR NOT NULL,'
               'PRIMARY KEY (movieid));')

### Check for tables

In [19]:
# List the tables within the Database
engine.table_names()

['Links', 'Movies', 'Ratings', 'Tags']

## LOAD

### Use pandas to load csv converted DataFrames into database

In [20]:
# Load the 'movies_df' DataFrame to the 'Movies' Table
movies_df.to_sql(name='Movies', con=engine, if_exists='append', index=False)

In [21]:
# Load the 'links_df' DataFrame to the 'Links' Table
links_df.to_sql(name='Links', con=engine, if_exists='append', index=False)

In [22]:
# Load the 'new_avg_ratings_df' DataFrame to the 'Ratings' table
new_avg_ratings_df.to_sql(name='Ratings', con=engine, if_exists='append', index=False)

In [23]:
# Load the 'new_tags_df' DataFrame to the 'Tags' table
new_tags_df.to_sql(name='Tags', con=engine, if_exists='append', index=False)

### We are creating a Final Table in the Database to store the final combined query results

In [24]:
# Create Table 'Movies' within the 'Movies_DB' database
engine.execute('CREATE TABLE "Movies_Final" ('
               'movieid VARCHAR NOT NULL,'
               'genres VARCHAR NOT NULL,'
               'title VARCHAR NOT NULL,'
               'release_year VARCHAR NOT NULL,'
               'rating FLOAT NOT NULL,'
               'imdbid VARCHAR NOT NULL,'
               'tmdbid VARCHAR NOT NULL,'
               'imdb_link VARCHAR NOT NULL,'
               'tmdb_link VARCHAR NOT NULL,'
               'tag VARCHAR NOT NULL,'
               'PRIMARY KEY (movieid));')

### Here, we are using INNER JOIN to join all the four tables and inserting the query results to the 'Movies_Final' table 

In [25]:
a = engine.execute('INSERT INTO Movies_Final '
                   'SELECT Movies.movieid, Movies.genres, Movies.title, Movies.release_year, Ratings.rating,'
                   ' Links.imdbid, Links.tmdbid, Links.imdb_link, Links.tmdb_link, Tags.tag '
                   'FROM Movies INNER JOIN Ratings on Movies.movieid = Ratings.movieid '
                   'INNER JOIN Links ON Movies.movieid = Links.movieid '
                   'INNER JOIN Tags ON Movies.movieid = Tags.movieid ')

In [26]:
# Connecting to the Database
cnx = sqlite3.connect('Movies_DB')

# Converting a Table in the Database into a Pandas DataFrame
df = pd.read_sql_query("SELECT * FROM Movies_Final", cnx)

### Displaying the Final Output in the Pandas DataFrame

In [27]:
df

,movieid,genres,title,release_year,rating,imdbid,tmdbid,imdb_link,tmdb_link,tag
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,3.886649,114709,862,https://www.imdb.com/title/tt0114709,https://www.themoviedb.org/movie/000862,"boy next door, family, disney, ya boy, BD-Vide..."
1,2,Adventure|Children|Fantasy,Jumanji,1995,3.246583,113497,8844,https://www.imdb.com/title/tt0113497,https://www.themoviedb.org/movie/008844,"see also:Zathura, giant insect, adapted from:b..."
2,3,Comedy|Romance,Grumpier Old Men,1995,3.173981,113228,15602,https://www.imdb.com/title/tt0113228,https://www.themoviedb.org/movie/015602,"moldy, comedinha de velhinhos engraÃƒÂ§ada, NO..."
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995,2.874540,114885,31357,https://www.imdb.com/title/tt0114885,https://www.themoviedb.org/movie/031357,"based on novel or book, interracial relationsh..."
4,5,Comedy,Father of the Bride Part II,1995,3.077291,113041,11862,https://www.imdb.com/title/tt0113041,https://www.themoviedb.org/movie/011862,"Touching, family, Steve Martin, it thought it ..."
